In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Import necessary libraries and make necessary arrangements
import time
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import xgboost 
import lightgbm as lgb
import warnings
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
warnings.filterwarnings('ignore')

In [3]:

# Date Features
def create_date_features(df):
    df['month'] = df.Date.dt.month
    df['day_of_month'] = df.Date.dt.day
    df['day_of_year'] = df.Date.dt.dayofyear
    df['week_of_year'] = df.Date.dt.weekofyear
    df['day_of_week'] = df.Date.dt.dayofweek
    df['year'] = df.Date.dt.year
    #df["is_wknd"] = df.Date.dt.weekday // 4
    #df['is_month_start'] = df.Date.dt.is_month_start.astype(int)
    #df['is_month_end'] = df.Date.dt.is_month_end.astype(int)
    return df

# Random Noise
def random_noise(dataframe):
    return np.random.normal(scale=2.0, size=(len(dataframe)))

# Lag/Shifted Features
def lag_features(dataframe, lags):
    for lag in lags:
        dataframe['sales_lag_' + str(lag)] = dataframe.groupby(["store_nbr", "family"])['sales'].transform(
            lambda x: x.shift(lag)) + random_noise(dataframe)
    return dataframe

# Rolling Mean Features
def roll_mean_features(dataframe, windows):
    for window in windows:
        dataframe['sales_roll_mean_' + str(window)] = dataframe.groupby(["store_nbr", "family"])['sales']. \
                                                          transform(
            lambda x: x.shift(16).rolling(window=window, min_periods=7, win_type="triang").mean()) + random_noise(
            dataframe)
    return dataframe

# Exponentially Weighted Mean Features
def ewm_features(dataframe, alphas, lags):
    for alpha in alphas:
        for lag in lags:
            dataframe['sales_ewm_alpha_' + str(alpha).replace(".", "") + "_lag_" + str(lag)] = \
                dataframe.groupby(["store_nbr", "family"])['sales'].transform(lambda x: x.shift(lag).ewm(alpha=alpha).mean())
    return dataframe


# Feature Importance
def plot_lgb_importances(model, plot=False, num=10):

    gain = model.feature_importance('gain')
    feat_imp = pd.DataFrame({'feature': model.feature_name(),
                             'split': model.feature_importance('split'),
                             'gain': 100 * gain / gain.sum()}).sort_values('gain', ascending=False)
    if plot:
        plt.figure(figsize=(10, 10))
        sns.set(font_scale=1)
        sns.barplot(x="gain", y="feature", data=feat_imp[0:25])
        plt.title('feature')
        plt.tight_layout()
        plt.show()
    else:
        print(feat_imp.head(num))
        

In [4]:
df_train = pd.read_csv(r"../data/train.csv", index_col=0)
df_test = pd.read_csv(r"../data/test.csv", index_col=0)
df_transactions = pd.read_csv(r"../data/transactions.csv")
df_stores = pd.read_csv(r"../data/stores.csv")
df_holidays = pd.read_csv(r"../data/holidays_events.csv")
df_oil = pd.read_csv(r"../data/oil.csv")

df_oil['date'] = df_oil['date'].astype("datetime64")
df_date = pd.DataFrame(pd.date_range("2013-01-01","2017-08-31"),columns=["date"])
df_oil = df_date.merge(df_oil,left_on='date',right_on='date',how='left').fillna(method='ffill').fillna(method='bfill')

In [5]:
df_all = df_train.append(df_test)
df_all = df_all.merge(df_stores,left_on="store_nbr",right_on="store_nbr",how="left")
df_holidays = df_holidays.rename(columns = {"type":"Holiday_type"})
df_all = df_all.merge(df_holidays,left_on="date",right_on="date",how="left")
df_all = df_all.merge(df_transactions,left_on=["date","store_nbr"],right_on=["date","store_nbr"],how="left")
df_all["date"] = df_all["date"].astype("datetime64")
df_all = df_all.merge(df_oil,left_on="date",right_on="date",how="left")
df_all = df_all.replace(",","_",regex=True)
df_all = df_all.rename(columns = {"date":"Date"})

In [6]:
col = ['Date',
       'store_nbr', 
       'family', 
       'sales', 
       'onpromotion', 
       #'city', 
       #'state',
       #'type', 
       'cluster', 
       'Holiday_type', 
       'locale', 
       #'locale_name',
       'description', 
       'transferred',
       'dcoilwtico'
      ]
df = df_all[col]

In [7]:
df.head()

,Date,store_nbr,family,sales,onpromotion,cluster,Holiday_type,locale,description,transferred,dcoilwtico
0,2013-01-01,1,AUTOMOTIVE,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14
1,2013-01-01,1,BABY CARE,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14
2,2013-01-01,1,BEAUTY,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14
3,2013-01-01,1,BEVERAGES,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14
4,2013-01-01,1,BOOKS,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14


In [8]:
df.loc[:,"store_nbr"] = df["store_nbr"].astype("category")
df.loc[:,"family"] = df["family"].astype("category")
df.loc[:,"onpromotion"] = df["onpromotion"].astype("int64")
#df.loc[:,"city"] = df["city"].astype("category")
#df.loc[:,"state"] = df["state"].astype("category")
#df.loc[:,"type"] = df["type"].astype("category")
df.loc[:,"Holiday_type"] = df["Holiday_type"].astype("category")
df.loc[:,"locale"] = df["locale"].astype("category")
#df.loc[:,"locale_name"] = df["locale_name"].astype("category")
df.loc[:,"description"] = df["description"].astype("category")
df.loc[:,"transferred"] = df["transferred"].astype("category")

In [9]:
zero_index = df.groupby(["store_nbr","family"]).sum()[df.groupby(["store_nbr","family"]).sum()["sales"]==0].index

## Data Engineering

In [10]:
df = create_date_features(df)
df

,Date,store_nbr,family,sales,onpromotion,cluster,Holiday_type,locale,description,transferred,dcoilwtico,month,day_of_month,day_of_year,week_of_year,day_of_week,year
0,2013-01-01,1,AUTOMOTIVE,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,1,2013
1,2013-01-01,1,BABY CARE,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,1,2013
2,2013-01-01,1,BEAUTY,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,1,2013
3,2013-01-01,1,BEVERAGES,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,1,2013
4,2013-01-01,1,BOOKS,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,1,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3082855,2017-08-31,9,POULTRY,NaN,1,6,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017
3082856,2017-08-31,9,PREPARED FOODS,NaN,0,6,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017
3082857,2017-08-31,9,PRODUCE,NaN,1,6,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017
3082858,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,6,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017


In [11]:
df = lag_features(df, lags = [16,17,18,19,20,21,22,30,31,90,180,365])
df

,Date,store_nbr,family,sales,onpromotion,cluster,Holiday_type,locale,description,transferred,dcoilwtico,month,day_of_month,day_of_year,week_of_year,day_of_week,year,sales_lag_16,sales_lag_17,sales_lag_18,sales_lag_19,sales_lag_20,sales_lag_21,sales_lag_22,sales_lag_30,sales_lag_31,sales_lag_90,sales_lag_180,sales_lag_365
0,2013-01-01,1,AUTOMOTIVE,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-01,1,BABY CARE,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-01-01,1,BEAUTY,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-01-01,1,BEVERAGES,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-01-01,1,BOOKS,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3082855,2017-08-31,9,POULTRY,NaN,1,6,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017,440.286253,285.071672,414.564259,382.439281,525.527382,292.185843,334.777782,569.180319,470.265980,552.106924,285.893724,414.565878
3082856,2017-08-31,9,PREPARED FOODS,NaN,0,6,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017,154.464227,115.130640,105.603103,130.437230,112.887273,113.037490,122.910312,47.104524,62.833953,73.104070,93.539926,131.055957
3082857,2017-08-31,9,PRODUCE,NaN,1,6,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017,2418.699828,1346.270948,1692.945294,1419.888770,1452.867717,1033.824435,1309.823978,2469.518448,1516.971516,1680.219832,1153.559497,1356.069536
3082858,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,6,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017,120.026127,179.430944,203.261489,141.452853,139.195377,147.362505,81.036723,204.974053,58.007038,2.443891,3.178916,83.880179


In [12]:
df = roll_mean_features(df,[16,17,18,30])
df

,Date,store_nbr,family,sales,onpromotion,cluster,Holiday_type,locale,description,transferred,dcoilwtico,month,day_of_month,day_of_year,week_of_year,day_of_week,year,sales_lag_16,sales_lag_17,sales_lag_18,sales_lag_19,sales_lag_20,sales_lag_21,sales_lag_22,sales_lag_30,sales_lag_31,sales_lag_90,sales_lag_180,sales_lag_365,sales_roll_mean_16,sales_roll_mean_17,sales_roll_mean_18,sales_roll_mean_30
0,2013-01-01,1,AUTOMOTIVE,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-01,1,BABY CARE,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-01-01,1,BEAUTY,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-01-01,1,BEVERAGES,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-01-01,1,BOOKS,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3082855,2017-08-31,9,POULTRY,NaN,1,6,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017,440.286253,285.071672,414.564259,382.439281,525.527382,292.185843,334.777782,569.180319,470.265980,552.106924,285.893724,414.565878,450.152182,457.222218,464.136157,445.718013
3082856,2017-08-31,9,PREPARED FOODS,NaN,0,6,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017,154.464227,115.130640,105.603103,130.437230,112.887273,113.037490,122.910312,47.104524,62.833953,73.104070,93.539926,131.055957,120.556862,119.136758,121.228460,101.024201
3082857,2017-08-31,9,PRODUCE,NaN,1,6,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017,2418.699828,1346.270948,1692.945294,1419.888770,1452.867717,1033.824435,1309.823978,2469.518448,1516.971516,1680.219832,1153.559497,1356.069536,1628.643495,1641.324032,1646.195464,1610.755255
3082858,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,6,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017,120.026127,179.430944,203.261489,141.452853,139.195377,147.362505,81.036723,204.974053,58.007038,2.443891,3.178916,83.880179,150.241550,146.957020,148.579514,87.565638


In [13]:
alphas = [0.95,0.9, 0.8, 0.5]
lags =[1, 7,30]
df = ewm_features(df, alphas, lags)
df

,Date,store_nbr,family,sales,onpromotion,cluster,Holiday_type,locale,description,transferred,dcoilwtico,month,day_of_month,day_of_year,week_of_year,day_of_week,year,sales_lag_16,sales_lag_17,sales_lag_18,sales_lag_19,sales_lag_20,sales_lag_21,sales_lag_22,sales_lag_30,sales_lag_31,sales_lag_90,sales_lag_180,sales_lag_365,sales_roll_mean_16,sales_roll_mean_17,sales_roll_mean_18,sales_roll_mean_30,sales_ewm_alpha_095_lag_1,sales_ewm_alpha_095_lag_7,sales_ewm_alpha_095_lag_30,sales_ewm_alpha_09_lag_1,sales_ewm_alpha_09_lag_7,sales_ewm_alpha_09_lag_30,sales_ewm_alpha_08_lag_1,sales_ewm_alpha_08_lag_7,sales_ewm_alpha_08_lag_30,sales_ewm_alpha_05_lag_1,sales_ewm_alpha_05_lag_7,sales_ewm_alpha_05_lag_30
0,2013-01-01,1,AUTOMOTIVE,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-01,1,BABY CARE,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-01-01,1,BEAUTY,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-01-01,1,BEVERAGES,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-01-01,1,BOOKS,0.0,0,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3082855,2017-08-31,9,POULTRY,NaN,1,6,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017,440.286253,285.071672,414.564259,382.439281,525.527382,292.185843,334.777782,569.180319,470.265980,552.106924,285.893724,414.565878,450.152182,457.222218,464.136157,445.718013,430.717553,430.717553,565.318991,423.935714,423.935714,560.611289,412.276337,412.276337,551.421713,392.438031,392.438031,518.514476
3082856,2017-08-31,9,PREPARED FOODS,NaN,0,6,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017,154.464227,115.130640,105.603103,130.437230,112.887273,113.037490,122.910312,47.104524,62.833953,73.104070,93.539926,131.055957,120.556862,119.136758,121.228460,101.024201,152.511953,152.511953,51.246066,150.443153,150.443153,52.444705,146.278339,146.278339,56.022566,134.243896,134.243896,72.911551
3082857,2017-08-31,9,PRODUCE,NaN,1,6,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017,2418.699828,1346.270948,1692.945294,1419.888770,1452.867717,1033.824435,1309.823978,2469.518448,1516.971516,1680.219832,1153.559497,1356.069536,1628.643495,1641.324032,1646.195464,1610.755255,2366.992548,2366.992548,2423.706728,2315.775659,2315.775659,2378.620204,2217.030205,2217.030205,2292.418673,1934.923106,1934.923106,2035.920473
3082858,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,6,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017,120.026127,179.430944,203.261489,141.452853,139.195377,147.362505,81.036723,204.974053,58.007038,2.443891,3.178916,83.880179,150.241550,146.957020,148.579514,87.565638,124.087264,124.087264,195.609900,127.218224,127.218224,188.128407,133.426673,133.426673,172.814546,148.242507,148.242507,122.237914


In [14]:
df['sales'] = np.log1p(df["sales"].values)

In [15]:
df_all["family"]=df_all["family"].astype("category")
df_all["store_nbr"]=df_all["store_nbr"].astype("category")
df_all["city"]=df_all["city"].astype("category")
df_all["state"]=df_all["state"].astype("category")
df_all["type"]=df_all["type"].astype("category")

## Train Test Split

In [16]:
df.tail(50)

,Date,store_nbr,family,sales,onpromotion,cluster,Holiday_type,locale,description,transferred,dcoilwtico,month,day_of_month,day_of_year,week_of_year,day_of_week,year,sales_lag_16,sales_lag_17,sales_lag_18,sales_lag_19,sales_lag_20,sales_lag_21,sales_lag_22,sales_lag_30,sales_lag_31,sales_lag_90,sales_lag_180,sales_lag_365,sales_roll_mean_16,sales_roll_mean_17,sales_roll_mean_18,sales_roll_mean_30,sales_ewm_alpha_095_lag_1,sales_ewm_alpha_095_lag_7,sales_ewm_alpha_095_lag_30,sales_ewm_alpha_09_lag_1,sales_ewm_alpha_09_lag_7,sales_ewm_alpha_09_lag_30,sales_ewm_alpha_08_lag_1,sales_ewm_alpha_08_lag_7,sales_ewm_alpha_08_lag_30,sales_ewm_alpha_05_lag_1,sales_ewm_alpha_05_lag_7,sales_ewm_alpha_05_lag_30
3082810,2017-08-31,8,HOME AND KITCHEN II,NaN,3,8,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017,40.156689,41.066961,56.305173,48.917498,44.540971,42.389204,79.774128,73.637731,58.629765,46.077693,51.070861,53.175601,49.387188,47.949480,50.409579,58.299419,3.529349e+01,3.529349e+01,7.316181e+01,3.566790e+01,3.566790e+01,7.225413e+01,3.662334e+01,3.662334e+01,7.027539e+01,4.063691e+01,4.063691e+01,65.150399
3082811,2017-08-31,8,HOME APPLIANCES,NaN,0,8,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017,-0.826854,3.661673,1.734095,1.531661,-1.288114,5.372727,-0.813358,-1.747601,-1.265223,2.830402,3.504800,0.617510,-4.827216,-3.522299,1.408156,-0.280100,4.656088e-41,4.656088e-41,7.628534e-23,9.991903e-32,9.991903e-32,9.991903e-18,2.136561e-22,2.136561e-22,1.304053e-12,4.589856e-10,4.589856e-10,0.000008
3082812,2017-08-31,8,HOME CARE,NaN,1,8,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017,322.510984,323.290050,347.306175,292.919165,308.098818,247.329677,271.816875,409.552791,419.443980,439.154771,361.845183,430.317395,331.522472,339.995993,348.780487,366.305454,3.211531e+02,3.211531e+02,4.077318e+02,3.213853e+02,3.213853e+02,4.087079e+02,3.219243e+02,3.219243e+02,4.111996e+02,3.210918e+02,3.210918e+02,415.581349
3082813,2017-08-31,8,LADIESWEAR,NaN,0,8,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017,32.987113,23.154402,33.743559,40.656262,25.383399,16.173136,22.765187,58.334837,32.414534,27.608350,52.949179,36.069470,26.284980,28.027220,29.562963,32.731675,3.066641e+01,3.066641e+01,5.661456e+01,3.037050e+01,3.037050e+01,5.525588e+01,2.991039e+01,2.991039e+01,5.259779e+01,2.935434e+01,2.935434e+01,44.426960
3082814,2017-08-31,8,LAWN AND GARDEN,NaN,0,8,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017,37.212134,30.566713,37.205239,31.308492,36.640405,39.057097,51.441051,22.505738,19.386599,22.495245,79.658124,19.507476,33.935347,36.298218,35.120795,33.103264,3.771676e+01,3.771676e+01,2.575331e+01,3.746424e+01,3.746424e+01,2.561593e+01,3.703660e+01,3.703660e+01,2.567697e+01,3.625075e+01,3.625075e+01,28.333893
3082815,2017-08-31,8,LINGERIE,NaN,1,8,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017,18.595660,23.407467,19.295992,21.830133,23.523370,35.145045,4.844589,53.662808,11.814510,24.175731,33.275634,26.878473,21.370558,20.891443,23.047521,22.416775,2.009490e+01,2.009490e+01,5.206349e+01,2.017941e+01,2.017941e+01,5.015770e+01,2.031951e+01,2.031951e+01,4.645637e+01,2.065651e+01,2.065651e+01,36.215336
3082816,2017-08-31,8,LIQUOR_WINE_BEER,NaN,2,8,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017,142.669981,79.866171,97.414747,203.700815,226.236356,193.957092,127.307613,101.885818,110.932367,80.616999,105.104620,153.463459,146.389128,146.005358,148.365340,144.498218,1.389109e+02,1.389109e+02,1.024177e+02,1.359979e+02,1.359979e+02,1.029289e+02,1.310298e+02,1.310298e+02,1.045365e+02,1.277252e+02,1.277252e+02,116.511782
3082817,2017-08-31,8,MAGAZINES,NaN,0,8,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017,22.349206,11.666452,21.814889,28.290097,9.033496,18.485238,24.730941,11.233376,16.420554,19.111018,7.042965,14.342810,21.315957,27.101061,20.711998,18.732159,1.962588e+01,1.962588e+01,1.025451e+01,1.930618e+01,1.930618e+01,1.051619e+01,1.883637e+01,1.883637e+01,1.105231e+01,1.864696e+01,1.864696e+01,12.763469
3082818,2017-08-31,8,MEATS,NaN,0,8,NaN,NaN,NaN,NaN,47.26,8,31,243,35,3,2017,362.361485,425.677407,420.

In [18]:
df['description'].unique().tolist()

['Primer dia del ano',
 nan,
 'Recupero puente Navidad',
 'Recupero puente primer dia del ano',
 'Carnaval',
 'Fundacion de Manta',
 'Provincializacion de Cotopaxi',
 'Fundacion de Cuenca',
 'Cantonizacion de Libertad',
 'Cantonizacion de Riobamba',
 'Viernes Santo',
 'Dia del Trabajo',
 'Dia de la Madre-1',
 'Cantonizacion del Puyo',
 'Dia de la Madre',
 'Batalla de Pichincha',
 'Cantonizacion de Guaranda',
 'Provincializacion de Imbabura',
 'Fundacion de Machala',
 'Cantonizacion de Latacunga',
 'Cantonizacion de El Carmen',
 'Fundacion de Santo Domingo',
 'Cantonizacion de Cayambe',
 'Fundacion de Guayaquil-1',
 'Fundacion de Guayaquil',
 'Fundacion de Esmeraldas',
 'Primer Grito de Independencia',
 'Fundacion de Riobamba',
 'Fundacion de Ambato',
 'Fundacion de Ibarra',
 'Cantonizacion de Quevedo',
 'Independencia de Guayaquil',
 'Traslado Independencia de Guayaquil',
 'Dia de Difuntos',
 'Independencia de Cuenca',
 'Provincializacion de Santo Domingo',
 'Provincializacion Santa El

In [31]:
df= df[df["Date"]>"2013-12-31"]

val = df[(df["Date"]>"2017-08-01")&(df["Date"]<="2017-08-15")]
train  = df[df["Date"]<="2017-08-01"]
test = df[df["Date"]>"2017-08-15"]

col_X = [col for col in train.columns if col not in ['Date', 'sales','year']]

y_train = train["sales"]
y_val = val["sales"]
X_train = train[col_X]
X_val = val[col_X]
X_test = test[col_X]
y_test = test["sales"]

## Train Model

In [32]:
# LightGBM parameters
lgb_params = {'metric': {'mse'},
              'boosting_type' : 'gbdt',
              'num_leaves': 8,
              'learning_rate': 0.2,
              #'feature_fraction': 0.8,
              'max_depth': 7,
              'verbose': 0,
              'num_boost_round': 5000,
              'early_stopping_rounds': 200,
              'nthread': -1,
             'force_col_wise':True}

lgbtrain = lgb.Dataset(data=X_train, label=y_train, feature_name=col_X)
lgbtest = lgb.Dataset(data=X_val, label=y_val, reference=lgbtrain, feature_name=col_X)

In [33]:
X_train

,store_nbr,family,onpromotion,cluster,Holiday_type,locale,description,transferred,dcoilwtico,month,day_of_month,day_of_year,week_of_year,day_of_week,sales_lag_16,sales_lag_17,sales_lag_18,sales_lag_19,sales_lag_20,sales_lag_21,sales_lag_22,sales_lag_30,sales_lag_31,sales_lag_90,sales_lag_180,sales_lag_365,sales_roll_mean_16,sales_roll_mean_17,sales_roll_mean_18,sales_roll_mean_30,sales_ewm_alpha_095_lag_1,sales_ewm_alpha_095_lag_7,sales_ewm_alpha_095_lag_30,sales_ewm_alpha_09_lag_1,sales_ewm_alpha_09_lag_7,sales_ewm_alpha_09_lag_30,sales_ewm_alpha_08_lag_1,sales_ewm_alpha_08_lag_7,sales_ewm_alpha_08_lag_30,sales_ewm_alpha_05_lag_1,sales_ewm_alpha_05_lag_7,sales_ewm_alpha_05_lag_30
657558,1,AUTOMOTIVE,0,13,Holiday,National,Primer dia del ano,False,98.17,1,1,1,1,2,0.529835,2.284376,0.708730,5.210110,3.877762,4.073425,1.074169,2.515256,-1.052063,0.954282,3.302942,7.422629,-0.064157,1.505815,2.079193,2.865655,1.948011,0.997531,2.850125,1.894159,0.990495,2.701010,1.793854,0.967697,2.408306,1.734309,0.995803,1.649229
657559,1,BABY CARE,0,13,Holiday,National,Primer dia del ano,False,98.17,1,1,1,1,2,-0.181434,2.530276,-2.007042,-0.008737,-0.816748,3.517063,-1.258204,-0.643785,-0.496745,-1.154643,-3.451016,1.616755,2.178659,-2.375046,0.495025,-1.746698,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
657560,1,BEAUTY,0,13,Holiday,National,Primer dia del ano,False,98.17,1,1,1,1,2,4.867686,0.666190,1.822694,2.100918,3.873711,-0.418717,2.417790,1.112978,0.414392,5.065818,1.268647,-2.640642,-0.032430,3.444366,2.575805,4.227790,1.092739,2.002488,4.750125,1.171847,2.009809,4.501009,1.294210,2.037045,4.008277,1.486519,2.137243,2.650041
657561,1,BEVERAGES,0,13,Holiday,National,Primer dia del ano,False,98.17,1,1,1,1,2,1150.576294,459.685187,1248.288448,1146.558262,1193.090644,1085.146819,1031.260963,989.487572,431.734695,843.394453,894.806742,1159.809824,879.461433,873.035361,864.273774,859.862116,846.224011,1257.132902,964.381324,849.093294,1262.965954,940.485374,846.694477,1265.199803,900.583271,817.460363,1214.463231,838.076597
657562,1,BOOKS,0,13,Holiday,National,Primer dia del ano,False,98.17,1,1,1,1,2,-1.341586,-1.477876,-1.422423,-0.232789,-2.937901,-0.582292,-1.549772,0.778080,-0.879925,1.627472,-5.361989,2.341772,1.863929,-0.780539,-3.726175,-0.427120,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,9,POULTRY,0,6,NaN,NaN,NaN,NaN,49.19,8,1,213,31,1,570.656547,519.014054,441.250087,316.665066,348.725122,322.177733,465.652318,514.382317,857.363984,542.262949,418.165727,451.798242,458.360139,467.451210,472.285322,487.391371,472.655814,319.198560,531.301019,474.348890,319.482360,547.758985,476.324564,323.205345,578.155552,466.832951,354.385849,625.841949
3029396,9,PREPARED FOODS,0,6,NaN,NaN,NaN,NaN,49.19,8,1,213,31,1,129.319188,144.219754,113.035236,95.595152,77.709682,105.740058,124.727282,136.678630,170.828755,150.697380,132.844699,91.264456,117.599445,122.561945,122.769641,129.685687,66.124368,53.874065,137.365348,70.280078,55.895064,138.982387,78.260842,60.058888,141.952373,95.360105,73.793706,145.723402
3029397,9,PRODUCE,151,6,NaN,NaN,NaN,NaN,49.19,8,1,213,31,1,2044.023184,1750.641241,1257.428670,1251.982213,1387.996046,2184.508753,1602.490512,1799.528348,2640.690694,1687.462024,1674.967468,1486.940696,1739.771418,1763.433211,1764.990145,1785.349664,1535.375566,2073.678593,1837.909370,1552.053040,2049.814028,1875.240158,1580.249364,2008.004903,1937.820042,1601.379946,1890.045546,1993.888392
3029398,9,SCHOOL AND OFFICE SUPPLIES,9,6,NaN,NaN,NaN,NaN,49.19,8,1,213,31,1,-0.035787,2.795543,0.551256,2.447644,-0.727402,1.683360,-3.761050,1.929152,5.646667,8.999936,5.745994,4.487391,-0.092136,4.153192,1.336870,3.877485,55.198005,4.850106,2.139887,54.284

In [34]:
y_train

657558     0.000000
657559     0.000000
657560     0.000000
657561     0.000000
657562     0.000000
             ...   
3029395    6.347732
3029396    3.940863
3029397    7.812565
3029398    5.318120
3029399    3.011409
Name: sales, Length: 2371842, dtype: float64

In [ ]:
from sklearn.metrics import mean_absolute_error
model = lgb.train(lgb_params, lgbtrain,
                  valid_sets=[lgbtrain, lgbtest],
                  num_boost_round=lgb_params['num_boost_round'],
                  early_stopping_rounds=lgb_params['early_stopping_rounds'],
                  #feval=mean_absolute_error,
                  verbose_eval=100,
                  )

In [ ]:
y_pred_val = model.predict(X_val, num_iteration=model.best_iteration)

In [ ]:
from sklearn.metrics import mean_squared_log_error

In [ ]:
y_pred_val[y_pred_val<0]=0

In [ ]:
mean_squared_log_error(np.expm1(y_val), np.expm1(y_pred_val))

In [ ]:
plt.scatter(np.expm1(y_val),np.expm1(y_pred_val))

In [ ]:
model.feature_importance()

In [ ]:
model.feature_name()

## Submission

In [ ]:
submission = pd.read_csv(r"C:\Users\derek.filko\DataspellProjects\Kaggle\Store Sales\data\sample_submission.csv")
y_test = model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
y_test[y_test<0]=0

In [ ]:
submission["sales"] = np.expm1(y_test)

In [ ]:
submission.to_csv("submission.csv",index=False)

In [ ]:
submission